In [7]:
!python -m pip show scikit-learn

Name: scikit-learn
Version: 1.2.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\users\erick\projects\course-mlops-zoomcamp\zoomcamp-env\lib\site-packages
Requires: threadpoolctl, numpy, joblib, scipy
Required-by: mlflow


In [8]:
import pickle
import pandas as pd

In [9]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

In [10]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [24]:
year, month = 2022, 2
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet')

In [25]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [26]:
std = y_pred.std()
print('Standard deviation:', std)

Standard deviation: 5.28140357655334


In [32]:
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

df_result = pd.DataFrame({'ride_id': df['ride_id'], 'predictions': y_pred})

output_file = 'predictions.parquet'

df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)

In [51]:
import os
print('Size of', output_file, ':', os.stat(output_file).st_size/1024, 'Kb')

Size of predictions.parquet : 58588.75 Kb
